In [1]:
import pandas as pd  # Importa a biblioteca pandas e a renomeia como pd para facilitar o acesso aos seus métodos e classes.
import numpy as np   # Importa a biblioteca numpy e a renomeia como np para facilitar o acesso aos seus métodos e classes.
import matplotlib.pyplot as plt  # Importa a subbiblioteca pyplot da biblioteca matplotlib e a renomeia como plt para facilitar o acesso aos seus métodos e funções de plotagem.
import re   # Importa o módulo re, que fornece operações com expressões regulares em Python.
import unicodedata   # Importa o módulo unicodedata, que fornece funções para trabalhar com caracteres Unicode.
from sklearn.preprocessing import minmax_scale  # Importa a função minmax_scale do módulo preprocessing da biblioteca sklearn.

In [2]:
dates = pd.Series(pd.date_range("2020-01-01", "2021-12-31"))  
# Cria uma série de datas usando pd.date_range() do pandas, começando em "2020-01-01" e terminando em "2021-12-31".

nome_do_arquivo_csv = "C:\\Users\\felipe.lucas\\Documents\\On-Semantically-Aware-Data-Science-An-Application-of-the-Disease-Ontology-DO-to-Clustering-COVID\\srag_adults_covid_hosp_2023-02-13.csv"  
# Define o caminho do arquivo CSV que será lido.

column_types = {'FATOR_RISC': str, 'DS_PCR_OUT': str, 'VACINA_COV': str, 'DOSE_1_COV': str, 'DOSE_2_COV': str, 'LAB_PR_COV': str, 'LOTE_1_COV': str, 'LOTE_2_COV': str}  
# Define um dicionário que mapeia os nomes das colunas do arquivo CSV para os tipos de dados que serão usados ao ler o arquivo.

srag_bruto = pd.read_csv(nome_do_arquivo_csv, dtype=column_types, low_memory = False)  
# Lê o arquivo CSV especificado usando pd.read_csv() do pandas, e aplica os tipos de dados definidos em column_types para cada coluna.

srag_bruto["date_int"] = pd.to_datetime(srag_bruto["date_int"])  
# Converte a coluna "date_int" do DataFrame srag_bruto para o tipo de dados datetime usando pd.to_datetime() do pandas.

srag_bruto = srag_bruto[srag_bruto["date_int"].isin(dates)]  
# Filtra o DataFrame srag_bruto para manter apenas as linhas em que a coluna "date_int" contém valores presentes na série de datas "dates".

# Seleção das colunas de estudo

In [3]:
srag_bruto.columns  # Retorna os nomes das colunas do DataFrame srag_bruto.

Index(['ID', 'date_not', 'SEM_NOT', 'SEM_PRI', 'date_sint', 'SG_UF_INTE',
       'REGIAO', 'CS_SEXO', 'NU_IDADE_N', 'FAIXA_IDADE', 'FAIXA_IDADE_SIMP',
       'CS_ESCOL_N', 'CS_RACA', 'NOSOCOMIAL', 'FEBRE_m', 'TOSSE_m',
       'GARGANTA_m', 'DISPNEIA_m', 'DESC_RESP_m', 'SATURACAO_m', 'DIARREIA_m',
       'VOMITO_m', 'OUTRO_SIN_m', 'OUTRO_DES', 'DOR_ABD', 'FADIGA',
       'PERD_OLFT', 'PERD_PALA', 'SRAG_original', 'SRAG_sfebre', 'PUERPERA_m',
       'FATOR_RISC', 'CARDIOPATI_m', 'HEMATOLOGI_m', 'SIND_DOWN_m',
       'HEPATICA_m', 'ASMA_m', 'DIABETES_m', 'NEUROLOGIC_m', 'PNEUMOPATI_m',
       'IMUNODEPRE_m', 'RENAL_m', 'OBESIDADE_m', 'OUT_MORBI_m', 'MORB_DESC',
       'CO_MUN_RES', 'SG_UF', 'REGIAO_RES', 'HOSPITAL', 'date_int',
       'CO_MU_INTE', 'UTI', 'SUPORT_VEN', 'RES_AN', 'RES_IGG', 'RES_IGM',
       'RES_IGA', 'AN_SARS2', 'PCR_RESUL', 'PCR_SARS2', 'DS_PCR_OUT',
       'CRITERIO', 'PCR', 'CLASSI_FIN', 'EVOLUCAO', 'date_desf', 'date_enc',
       'n_comorb_m', 'n_comorb_mreal', 'CONT

#Retirar todos os pacientes "Ongoing" da lista, visto que, o desdobramento de seus casos são inconclusivos.

In [4]:
srag_bruto = srag_bruto[srag_bruto['EVOLUCAO'] != "Ongoing"] # Todos os pacientes "Ongoing" foram retirados do DataFramez

#Conferindo se todos os pacientes estão enquadrados na categoria de "COVID19"

In [5]:
srag_bruto["CLASSI_FIN"].value_counts() #Confere todas as classes pertences na coluna "CLASSI_FIN"

CLASSI_FIN
5.0    1671979
Name: count, dtype: int64

In [6]:
srag_bruto['IS_DEATH'] = srag_bruto['EVOLUCAO'].apply(lambda x: 1 if x== "Death" else 0)
# Adiciona uma nova coluna chamada 'IS_DEATH' ao DataFrame srag_bruto. O valor será 1 se o valor na coluna 'EVOLUCAO' for "Death" e 0 caso contrário.

In [7]:
colunas = ["ID","date_int","CS_SEXO",'NU_IDADE_N', 'CARDIOPATI_m', 'HEMATOLOGI_m', 'SIND_DOWN_m',
       'HEPATICA_m', 'ASMA_m', 'DIABETES_m', 'NEUROLOGIC_m', 'PNEUMOPATI_m',
       'IMUNODEPRE_m', 'RENAL_m', 'OBESIDADE_m', 'IS_CAPITAL',  'CO_MUN_RES','CO_MU_INTE','UTI','IS_DEATH','CS_RACA']
srag_bruto = srag_bruto[colunas]
# Define uma lista chamada 'colunas' contendo os nomes das colunas que desejamos manter no DataFrame srag_bruto.

In [8]:
srag_bruto #Imprime o DataFrame

,ID,date_int,CS_SEXO,NU_IDADE_N,CARDIOPATI_m,HEMATOLOGI_m,SIND_DOWN_m,HEPATICA_m,ASMA_m,DIABETES_m,...,PNEUMOPATI_m,IMUNODEPRE_m,RENAL_m,OBESIDADE_m,IS_CAPITAL,CO_MUN_RES,CO_MU_INTE,UTI,IS_DEATH,CS_RACA
0,1,2020-03-24,Male,57,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Yes,355030.0,355030.0,No,0,NaN
1,2,2020-03-23,Female,61,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Yes,355030.0,355030.0,No,0,NaN
2,3,2020-03-23,Female,92,Yes,No,No,No,No,No,...,No,No,No,No,Yes,431490.0,431490.0,No,1,White
3,4,2020-03-26,Male,60,NaN,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,NaN,NaN,Yes,355030.0,355030.0,Yes,1,White
4,5,2020-03-13,Female,54,Yes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Yes,Yes,355030.0,355030.0,Yes,1,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995675,2083081,2021-11-05,Male,68,Yes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Yes,355030.0,355030.0,Yes,1,NaN
1997179,2084652,2021-12-15,Male,67,Yes,NaN,NaN,NaN,NaN,Yes,...,NaN,NaN,Yes,NaN,Yes,431490.0,431490.0,NaN,0,White
1997356,2084837,2021-12-27,Female,63,No,No,No,No,No,No,...,No,No,No,No,NaN,355030.0,NaN,No,0,White
1997962,2085468,2021-12-27,Male,33,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,410900.0,411370.0,No,0,White


# Identificação dos dados faltantes em cada coluna

In [9]:
srag_bruto.isnull().sum() # Calcula o número de valores ausentes em cada coluna do DataFrame srag_bruto.

ID                    0
date_int              0
CS_SEXO             170
NU_IDADE_N            0
CARDIOPATI_m     843454
HEMATOLOGI_m    1047860
SIND_DOWN_m     1048369
HEPATICA_m      1049314
ASMA_m          1039258
DIABETES_m       901052
NEUROLOGIC_m    1034279
PNEUMOPATI_m    1034023
IMUNODEPRE_m    1042700
RENAL_m         1034903
OBESIDADE_m     1010590
IS_CAPITAL         2387
CO_MUN_RES          236
CO_MU_INTE         2387
UTI              158865
IS_DEATH              0
CS_RACA          302102
dtype: int64

In [10]:
porcentagem_faltante = (srag_bruto.isnull().sum() / len(srag_bruto)) * 100 # Calcula a porcentagem de valores ausentes em cada coluna do DataFrame srag_bruto.
print(porcentagem_faltante)

ID               0.000000
date_int         0.000000
CS_SEXO          0.010168
NU_IDADE_N       0.000000
CARDIOPATI_m    50.446447
HEMATOLOGI_m    62.671840
SIND_DOWN_m     62.702283
HEPATICA_m      62.758803
ASMA_m          62.157360
DIABETES_m      53.891347
NEUROLOGIC_m    61.859569
PNEUMOPATI_m    61.844258
IMUNODEPRE_m    62.363223
RENAL_m         61.896890
OBESIDADE_m     60.442745
IS_CAPITAL       0.142765
CO_MUN_RES       0.014115
CO_MU_INTE       0.142765
UTI              9.501615
IS_DEATH         0.000000
CS_RACA         18.068528
dtype: float64


# Binarização das colunas categóricas

#De maneira geral, as tipologias de coluna "object" e "category" se enquadram na premissa de "Categóricas", enquanto "int64" e "float64" se enquadram em "contínuas".

In [11]:
srag_bruto.dtypes # Retorna os tipos de dados de cada coluna no DataFrame srag_bruto.

ID                       int64
date_int        datetime64[ns]
CS_SEXO                 object
NU_IDADE_N               int64
CARDIOPATI_m            object
HEMATOLOGI_m            object
SIND_DOWN_m             object
HEPATICA_m              object
ASMA_m                  object
DIABETES_m              object
NEUROLOGIC_m            object
PNEUMOPATI_m            object
IMUNODEPRE_m            object
RENAL_m                 object
OBESIDADE_m             object
IS_CAPITAL              object
CO_MUN_RES             float64
CO_MU_INTE             float64
UTI                     object
IS_DEATH                 int64
CS_RACA                 object
dtype: object

#Como é possível observar, as categorias candidatas a binarização são: CS_SEXO; CARDIOPATI_m; HEMATOLOGI_m; SIND_DOWN_m;     HEPATICA_m; ASMA_m; DIABETES_m; NEUROLOGIC_m; PNEUMOPATI_m; IMUNODEPRE_m; RENAL_m; OBESIDADE_m; MORB_DESC; IS_CAPITAL.  

In [12]:
Semelhantes = ["CARDIOPATI_m", "HEMATOLOGI_m", "SIND_DOWN_m", "HEPATICA_m", "ASMA_m", "DIABETES_m", "NEUROLOGIC_m", "PNEUMOPATI_m", "IMUNODEPRE_m", "RENAL_m", "OBESIDADE_m","UTI",'IS_CAPITAL']
# Lista de colunas que são semelhantes e devem ser convertidas para valores binários (0 ou 1).
srag_bruto[Semelhantes] = srag_bruto[Semelhantes].applymap(lambda x: 1 if x== "Yes" else 0) 
# Converte os valores "Yes" para 1 e qualquer outro valor para 0 nas colunas especificadas no DataFrame srag_bruto.

In [13]:
srag_bruto["CS_SEXO"] = srag_bruto["CS_SEXO"].apply(lambda x: 1 if x== "Male" else 0) 
# Converte os valores "Male" para 1 e "Female" para 0 na coluna "CS_SEXO" do DataFrame srag_bruto.

#Para esse próximo cenário é preciso criar novas colunas e gerar interações com outras já pré exixtentes. 

In [14]:
srag_bruto["White"] = np.where(srag_bruto["CS_RACA"]== "White", 1, 0) 
# Conta o número de ocorrências de cada valor na coluna "CS_RACA" do DataFrame srag_bruto.

In [15]:
srag_bruto["Black/Brown"] = np.where(srag_bruto["CS_RACA"]== "Black/Brown", 1, 0) 
# Cria uma nova coluna chamada "White" no DataFrame srag_bruto, atribuindo 1 se o valor na coluna "CS_RACA" for "White" e 0 caso contrário.

In [16]:
srag_bruto["Asian"] = np.where(srag_bruto["CS_RACA"]== "Asian", 1, 0)  
# Cria uma nova coluna chamada "Asian" no DataFrame srag_bruto, atribuindo 1 se o valor na coluna "CS_RACA" for "Asian" e 0 caso contrário.

In [17]:
srag_bruto["Indigenous"] = np.where(srag_bruto["CS_RACA"]== "Indigenous", 1, 0)  
# Cria uma nova coluna chamada "Indigenous" no DataFrame srag_bruto, atribuindo 1 se o valor na coluna "CS_RACA" for "Indigenous" e 0 caso contrário

In [18]:
srag_bruto = srag_bruto.drop("CS_RACA", axis = 1) # Remove a coluna "CS_RACA" do DataFrame srag_bruto.

In [19]:
srag_bruto #Imprime o Dataframe srag_bruto

,ID,date_int,CS_SEXO,NU_IDADE_N,CARDIOPATI_m,HEMATOLOGI_m,SIND_DOWN_m,HEPATICA_m,ASMA_m,DIABETES_m,...,OBESIDADE_m,IS_CAPITAL,CO_MUN_RES,CO_MU_INTE,UTI,IS_DEATH,White,Black/Brown,Asian,Indigenous
0,1,2020-03-24,1,57,0,0,0,0,0,0,...,0,1,355030.0,355030.0,0,0,0,0,0,0
1,2,2020-03-23,0,61,0,0,0,0,0,0,...,0,1,355030.0,355030.0,0,0,0,0,0,0
2,3,2020-03-23,0,92,1,0,0,0,0,0,...,0,1,431490.0,431490.0,0,1,1,0,0,0
3,4,2020-03-26,1,60,0,0,0,0,0,1,...,0,1,355030.0,355030.0,1,1,1,0,0,0
4,5,2020-03-13,0,54,1,0,0,0,0,0,...,1,1,355030.0,355030.0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995675,2083081,2021-11-05,1,68,1,0,0,0,0,0,...,0,1,355030.0,355030.0,1,1,0,0,0,0
1997179,2084652,2021-12-15,1,67,1,0,0,0,0,1,...,0,1,431490.0,431490.0,0,0,1,0,0,0
1997356,2084837,2021-12-27,0,63,0,0,0,0,0,0,...,0,0,355030.0,NaN,0,0,1,0,0,0
1997962,2085468,2021-12-27,1,33,0,0,0,0,0,0,...,0,0,410900.0,411370.0,0,0,1,0,0,0


# Escalonamento das colunas contínuas

#As colunas candidatas são: NU_IDADE_N, CO_MUM_RES e CO_MU_INTE

In [20]:
Semelhantes_02 = ["NU_IDADE_N"] # Lista de colunas que são semelhantes e devem ser escalonadas com MinMaxScaler.
srag_bruto[Semelhantes_02] = minmax_scale(srag_bruto[Semelhantes_02]) 
# Aplica a função minmax_scale para escalonar as colunas especificadas no DataFrame srag_bruto.

# Importação de novo DataFrame. Nesse caso, um arquivo do IBGE contendo o IDH dos municípios brasileiros.

In [21]:
nome_do_arquivo_csv_02 = "C:\\Users\\felipe.lucas\\Downloads\\AtlasBrasil_Consulta.xlsx" # Importa um arquivo da máquina
IBGE_IDH = pd.read_excel(nome_do_arquivo_csv_02) # Efetua a leitura do arquivo em formato Excel
IBGE_IDH["code"] = IBGE_IDH["code"]//10 # Realiza a divisão de todos os algoritmos da coluna por 10
IBGE_IDH = IBGE_IDH.drop("Espacialidades", axis = 1) # Retira a coluna selecionada do DataFrame
IBGE_IDH = IBGE_IDH.set_index("code")["IDHM2010"].to_dict() # Seleciona as colunas que assumem o papel de chave e valor para a confecção de um dicionário
IBGE_IDH # Imprime o dicionário 

{7: 0.727,
 520005: 0.708,
 310010: 0.689,
 520010: 0.689,
 310020: 0.698,
 150010: 0.628,
 230010: 0.628,
 290010: 0.603,
 290020: 0.575,
 410010: 0.687,
 420005: 0.694,
 420010: 0.696,
 150013: 0.622,
 310030: 0.654,
 260005: 0.679,
 170025: 0.665,
 310040: 0.63,
 210005: 0.672,
 290030: 0.582,
 150020: 0.506,
 230015: 0.606,
 230020: 0.601,
 240010: 0.679,
 220005: 0.528,
 430003: 0.687,
 230030: 0.595,
 510010: 0.628,
 120001: 0.604,
 520013: 0.686,
 240020: 0.661,
 310050: 0.61,
 350010: 0.79,
 520015: 0.702,
 350020: 0.73,
 410020: 0.667,
 290035: 0.546,
 260010: 0.657,
 240030: 0.585,
 320010: 0.667,
 210010: 0.529,
 260020: 0.588,
 150030: 0.489,
 260030: 0.592,
 220010: 0.599,
 420020: 0.725,
 420030: 0.741,
 150034: 0.564,
 310060: 0.576,
 510020: 0.729,
 220020: 0.639,
 250010: 0.572,
 270010: 0.549,
 500020: 0.67,
 310070: 0.675,
 420040: 0.698,
 210015: 0.5,
 320016: 0.652,
 290040: 0.55,
 520017: 0.671,
 350030: 0.715,
 520020: 0.722,
 310080: 0.663,
 240040: 0.616,
 2600

#Criando as colunas "IDH_MUM_RES" e "IDH_MU_INTE"

In [22]:
srag_bruto["IDH_MUM_RES"] = srag_bruto["CO_MUN_RES"].map(IBGE_IDH) # A nova coluna criada usa os valores do dicionários se caso a chave acionada tiver o código compatível.
srag_bruto["IDH_MU_INTE"] = srag_bruto["CO_MU_INTE"].map(IBGE_IDH)  # A nova coluna criada usa os valores do dicionários se caso a chave acionada tiver o código compatível.

In [23]:
srag_bruto[["IDH_MUM_RES","IDH_MU_INTE"]] = minmax_scale(srag_bruto[["IDH_MUM_RES","IDH_MU_INTE"]])

In [24]:
srag_bruto["IDH_MUM_RES"].fillna(srag_bruto["IDH_MUM_RES"].mean(), inplace = True)
srag_bruto["IDH_MU_INTE"].fillna(srag_bruto["IDH_MU_INTE"].mean(), inplace = True)
srag_bruto[["IDH_MUM_RES","IDH_MU_INTE"]]

,IDH_MUM_RES,IDH_MU_INTE
0,0.871622,0.871622
1,0.871622,0.871622
2,0.871622,0.871622
3,0.871622,0.871622
4,0.871622,0.871622
...,...,...
1995675,0.871622,0.871622
1997179,0.871622,0.871622
1997356,0.871622,0.779259
1997962,0.639640,0.810811


#Criando a coluna "SAME_MUM" que tem por objetivo avaliar se os municípios de internação e residencial são semelhantes 

In [25]:
srag_bruto['mesmo_mun'] = (srag_bruto["CO_MUN_RES"] == srag_bruto["CO_MU_INTE"]).map(int)

In [26]:
#srag_bruto['mesmo_mun'].fillna("nulo").value_counts()

In [27]:
srag_bruto.drop(["date_int", "CO_MUN_RES", "CO_MU_INTE"], inplace = True, axis = 1)
srag_bruto

,ID,CS_SEXO,NU_IDADE_N,CARDIOPATI_m,HEMATOLOGI_m,SIND_DOWN_m,HEPATICA_m,ASMA_m,DIABETES_m,NEUROLOGIC_m,...,IS_CAPITAL,UTI,IS_DEATH,White,Black/Brown,Asian,Indigenous,IDH_MUM_RES,IDH_MU_INTE,mesmo_mun
0,1,1,0.373737,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0.871622,0.871622,1
1,2,0,0.414141,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0.871622,0.871622,1
2,3,0,0.727273,1,0,0,0,0,0,1,...,1,0,1,1,0,0,0,0.871622,0.871622,1
3,4,1,0.404040,0,0,0,0,0,1,0,...,1,1,1,1,0,0,0,0.871622,0.871622,1
4,5,0,0.343434,1,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0.871622,0.871622,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995675,2083081,1,0.484848,1,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0.871622,0.871622,1
1997179,2084652,1,0.474747,1,0,0,0,0,1,0,...,1,0,0,1,0,0,0,0.871622,0.871622,1
1997356,2084837,0,0.434343,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0.871622,0.779259,0
1997962,2085468,1,0.131313,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0.639640,0.810811,0


In [28]:
srag_bruto.columns

Index(['ID', 'CS_SEXO', 'NU_IDADE_N', 'CARDIOPATI_m', 'HEMATOLOGI_m',
       'SIND_DOWN_m', 'HEPATICA_m', 'ASMA_m', 'DIABETES_m', 'NEUROLOGIC_m',
       'PNEUMOPATI_m', 'IMUNODEPRE_m', 'RENAL_m', 'OBESIDADE_m', 'IS_CAPITAL',
       'UTI', 'IS_DEATH', 'White', 'Black/Brown', 'Asian', 'Indigenous',
       'IDH_MUM_RES', 'IDH_MU_INTE', 'mesmo_mun'],
      dtype='object')

# Salvando csv base limpa

In [29]:
srag_bruto.to_csv("srag_tratada.csv", index = False)

## NLP

In [30]:
# srag_bruto["MORB_DESC"].value_counts() # Conta o número de ocorrências de cada valor na coluna "MORB_DESC" do DataFrame srag_bruto.

In [31]:
# srag_bruto["MORB_DESC"] = srag_bruto["MORB_DESC"].str.replace('[^a-zA-Z0-9]', ',', regex=True) #Substitui todos os caracteres não alfanuméricos por ","

# Premissa a ser resolvida posteriormente

In [32]:
# srag_bruto["date_int"] = (pd.to_datetime("2021-12-31") - srag_bruto["date_int"]).dt.days 
# Calcula a diferença de dias entre cada data na coluna "date_int" e a data '2021-12-31', e substitui os valores na coluna "date_int" por essa diferença em dias.

In [33]:
# srag_bruto["date_int"] = minmax_scale(srag_bruto["date_int"]) # Aplica a função minmax_scale para escalonar os valores na coluna "date_int" do DataFrame srag_bruto.